<a href="https://colab.research.google.com/github/alima-parveen/synthetic_data_generation-/blob/main/Synthetic_Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
!{sys.executable} -m pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Load your datasets
products_df = pd.read_csv('/content/drive/MyDrive/assignment_reviews_metadata/product_asin.csv')

# Sample 1000 products only to reduce computation
sampled_products_df = products_df.sample(n=1000)


print("\nTop 10 sampled products:")
print(sampled_products_df.head(10))


Top 10 sampled products:
             X                                              title parent_asin  \
152903  253165  Nature's Way Premium Herbal Neem Leaf 950 mg, ...  B001E11DMI   
341384  561646  Luna Body Kinesiology Tape Precut, Elastic Spo...  B074MYYJPD   
101696  168820  Lavender Sleep Spray, Pillow Mist with Pure La...  B097462P2G   
103866  172448  Mepiform Self-Adherent Silicone Dressing 1.6x1...  B00IG11JR8   
384270  631442  Hot Cream 2 Pack, Fat Burning Cream, Body Slim...  B087R4VWYG   
395483  649770  Pedag 132 Girl Self Adhesive Forefoot Grip for...  B0026FDCUW   
84711   141176  URPOWER OD-A001 Real Bamboo Essential Oil Ultr...  B073F4R7GR   
441432  725420  Peppermint Essential Oil - 100% Pure Uncut Und...  B01N9IJVHL   
158803  262909  Vagisil Scentsitive Scents Feminine Dry Wash D...  B0C6HGK623   
420934  691365  High Decibel Ear Plugs Noise Cancelling Ear Pr...  B07ZL1Y6V6   

                   categories                             cat1  \
152903  ['Health

In [4]:
import os
from typing import Dict, List
from groq import Groq

# Get a free API key from https://console.groq.com/keys
os.environ["GROQ_API_KEY"] = "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"

LLAMA3_405B_INSTRUCT = "llama-3.1-405b-reasoning"
LLAMA3_70B_INSTRUCT = "llama-3.1-70b-versatile"
LLAMA3_8B_INSTRUCT = "llama3.1-8b-instant"

DEFAULT_MODEL = LLAMA3_70B_INSTRUCT

In [15]:
# Function to generate the prompt for title
def generate_title_prompt(product_title, categories, rating):
    if rating >= 4:
        return f"Write a short and catchy title summarizing a positive review for a {categories} product called {product_title} in not more than 10 words. Should not include words like Title or Review as Starting words."
    elif rating <= 2:
        return f"Write a short and catchy title summarizing a negative review for a {categories} product called {product_title} in not more than 10 words. Should not include words like Title or Review as Starting words."
    else:
        return f"Write a short and catchy title summarizing a mixed review for a {categories} product called {product_title} in not more than 10 words. Should not include words like Title or Review as Starting words."

In [5]:
# Function to generate the prompt for review
def generate_review_prompt(product_title, categories, rating):
    if rating >= 4:
        return f"Write a positive review for a {categories} product called {product_title} in not more than 75 words paragraph closer to human review and it should include benefits"
    elif rating <= 2:
        return f"Write a negative review for a {categories} product called {product_title} in not more than 75 words paragraph closer to human review. Mention any issues or side effects."
    else:
        return f"Write a mixed review for a {categories} product called {product_title} in not more than 75 words paragraph closer to human review. Mention both pros and cons."

In [24]:
# Function to generate the text using llama3
def generate_review(product_title, categories, rating):
    client = Groq()
    review_prompt = generate_review_prompt(product_title, categories, rating)
    title_prompt = generate_title_prompt(product_title, categories, rating)
    prompt = title_prompt + "." + review_prompt
    print(f"Prompt = {prompt}")
    completion = client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.7,
        max_tokens=150,
        top_p=1,
        stream=True,
        stop=None,
    )
    response = ""
    for chunk in completion:
      response += chunk.choices[0].delta.content or ""
      #print(response, end="")
    print(f"\nResponse = {response}")

    return response

In [25]:
# Function to generate the review using sampled products
def generate_synthetic_reviews(num_reviews):
    synthetic_data = []
    for _ in range(num_reviews):
        product = sampled_products_df.sample(1).iloc[0]
        rating = random.randint(1, 5)

        review_text = generate_review(product['title'], product['cat1'], rating)
        title = review_text.split('\n')[0].strip()  # Extract the first sentence as title
        title = title.replace("**", "").strip()
        review_text = review_text.replace("**", "").strip()
        review_text = ''.join(review_text.split('\n')[1:]).strip()
        review_data = {
            'rating': rating,
            'title': title,
            'text': review_text,
            'asin': product['parent_asin'],
            'parent_asin': product['parent_asin'],
            'user_id': f"user_{random.randint(1000, 9999)}",
            'timestamp': datetime.now() - timedelta(days=random.randint(1, 365)),
            'helpful_vote': random.randint(0, 100),
            'verified_purchase': random.choice([True, False]),
            'date': (datetime.now() - timedelta(days=random.randint(1, 365))).strftime('%Y-%m-%d'),
            'time': datetime.now().strftime('%H:%M:%S')
        }

        synthetic_data.append(review_data)

    return pd.DataFrame(synthetic_data)

In [26]:
# Generate 1000 synthetic reviews
synthetic_reviews_df = generate_synthetic_reviews(1000)
synthetic_reviews_df.to_csv('/content/drive/MyDrive/assignment_reviews_metadata/synthetic_reviews_10.csv', index=False)
print("Synthetic data generated and saved to 'synthetic_reviews_1000.csv'")

Streaming output truncated to the last 5000 lines.

Response = Decent Support with Some Drawbacks.

I've been using the BERTER Knee Brace for a few weeks now, and it's been a mixed bag. On the plus side, it provides decent support and stability, which has helped alleviate some of my knee pain. The neoprene material is also breathable and comfortable. However, the sizing can be a bit off, and the straps can dig into your skin if not adjusted properly. Overall, it's not perfect, but it's worth considering for mild to moderate knee issues.
Prompt = Write a short and catchy title summarizing a negative review for a  'Health Care' product called Shortie's Candle Company True Rose Soy Wax Melts 3 Pack - 3 Highly Scented 3 Oz. Bars - Made with 100% Soy and Essential Fragrance Oils - Phthalate & Paraffin Free, Vegan, Non-Toxic in not more than 10 words. Should not include words like Title or Review as Starting words..Write a negative review for a  'Health Care' product called Shortie's Candle 